In [ ]:
import json
import matplotlib
from matplotlib import pyplot as plt
import pickle
import sklearn
import numpy as np
import statistics
%matplotlib inline

In [ ]:
oresults_path = '../predicted-results/original/nbins-10'
mresults_path = '../predicted-results/mixed5050/nbins-10'
eresults_path = '../predicted-results/everything/nbins-10'

omodels_path = '../models/original'
mmodels_path = '../models/mixed5050/nbins-10'
emodels_path = '../models/everything/nbins-10'

test_sets = ['5050', '2575', 'everything']

mpoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 'all']
epoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 4000, 6000, 'all']


# Calculating Error

In [ ]:
# Consider 5050 models and everything models at the 2500 mark 
m5models = dict()
emodels = dict()
omodels = dict()
# 5050 model
for tset in ['5050', '2575', 'everything']:
    m5models[tset] = dict()
    emodels[tset] = dict()
    omodels[tset] = dict()
    
    '''First load the original models'''
    for target in ['COF', 'intercept']:
        with open(f'{omodels_path}/{target}.pickle', 'rb') as f:
            model = pickle.load(f)
        with open(f'{omodels_path}/{target}.pickle', 'rb') as f:
            features = pickle.load(f)
        with open(f'{oresults_path}/{target}_on_{tset}.json', 'r') as f:
            data = json.load(f)
        omodels[tset][target] = {'model': model,
                                 'features': features,
                                 'data': data,
                                 'n_train': len(model.oob_prediction_),
                                 'r_square': data[target]['r_square']}
        
    '''Then load the mixed5050 models'''
    for point in mpoints:
        for i in range(5):
            for target in ['COF', 'intercept']:
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{mresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                if i == 0:
                    if not m5models[tset].get(target):
                        m5models[tset][target] = dict()
                    m5models[tset][target][point] = {
                      'model': model,
                      'features': features,
                      'data': [data],
                      'n_train': len(model.oob_prediction_),
                      'r_square': [data[target]['r_square']]}
                else:
                    m5models[tset][target][point]['data'].append(data)
                    m5models[tset][target][point]['r_square'].append(data[target]['r_square'])
                    
    '''Finally load the combined models'''
    for point in epoints:
        # Lastly deal with the everything models
        for i in range(5):
            for target in ['COF', 'intercept']:
                with open(f'{emodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{emodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{eresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                if i == 0:
                    if not emodels[tset].get(target):
                        emodels[tset][target] = dict()
                    emodels[tset][target][point] = {
                      'model': model,
                      'features': features,
                      'data': [data],
                      'n_train': len(model.oob_prediction_),
                      'r_square': [data[target]['r_square']]}
                else:
                    emodels[tset][target][point]['data'].append(data)
                    emodels[tset][target][point]['r_square'].append(data[target]['r_square'])


In [ ]:
for tset in ['5050', '2575', 'everything']:
    for point in mpoints:
        for target in ['COF', 'intercept']:

            m5models[tset][target][point]['ave_data'] = dict() 
            m5models[tset][target][point]['std_data'] = dict()
            m5models[tset][target][point]['ratio-to-simulated'] = dict()
            m5models[tset][target][point]['error'] = dict()
            for j in m5models[tset][target][point]['data'][0][target]:
                for i in range(5):
                    if i==0 and j!='r_square':
                        m5models[tset][target][point]['ave_data'][j] = [m5models[tset][target][point]['data'][i][target][j][f'predicted-{target}']]
                    elif j!='r_square':
                        m5models[tset][target][point]['ave_data'][j].append(m5models[tset][target][point]['data'][i][target][j][f'predicted-{target}'])
                if j!='r_square':
                    m5models[tset][target][point]['std_data'][j] = statistics.stdev(m5models[tset][target][point]['ave_data'][j])
                    m5models[tset][target][point]['ave_data'][j] = statistics.mean(m5models[tset][target][point]['ave_data'][j])
                    m5models[tset][target][point]['ratio-to-simulated'][j] = m5models[tset][target][point]['ave_data'][j]/m5models[tset][target][point]['data'][i][target][j][f'simulated-{target}']
                    m5models[tset][target][point]['error'][j] = m5models[tset][target][point]['ave_data'][j] - m5models[tset][target][point]['data'][i][target][j][f'simulated-{target}']
    for point in epoints:
        for target in ['COF', 'intercept']:
            emodels[tset][target][point]['ave_data'] = dict()
            emodels[tset][target][point]['std_data'] = dict()
            emodels[tset][target][point]['ratio-to-simulated'] = dict()
            emodels[tset][target][point]['error'] = dict()
            for j in emodels[tset][target][point]['data'][0][target]:
                for i in range(5):
                    if i==0 and j!='r_square':
                        emodels[tset][target][point]['ave_data'][j] = [emodels[tset][target][point]['data'][i][target][str(j)][f'predicted-{target}']]
                    elif j!='r_square':
                        emodels[tset][target][point]['ave_data'][j].append(emodels[tset][target][point]['data'][i][target][str(j)][f'predicted-{target}'])
                if j!='r_square':
                    emodels[tset][target][point]['std_data'][j] = statistics.stdev(emodels[tset][target][point]['ave_data'][j])
                    emodels[tset][target][point]['ave_data'][j] = statistics.mean(emodels[tset][target][point]['ave_data'][j])
                    emodels[tset][target][point]['ratio-to-simulated'][j] = emodels[tset][target][point]['ave_data'][j]/emodels[tset][target][point]['data'][i][target][j][f'simulated-{target}']
                    emodels[tset][target][point]['error'][j] = emodels[tset][target][point]['ave_data'][j] - emodels[tset][target][point]['data'][i][target][j][f'simulated-{target}']
    for target in ['COF', 'intercept']:
        omodels[tset][target]['ratio-to-simulated'] = dict()
        omodels[tset][target]['error'] = dict()
        for j in omodels[tset][target]['data'][target]:
            if j!='r_square':
                omodels[tset][target]['ratio-to-simulated'][j] = omodels[tset][target]['data'][target][j][f'predicted-{target}']/omodels[tset][target]['data'][target][j][f'simulated-{target}']
                omodels[tset][target]['error'][j] = omodels[tset][target]['data'][target][j][f'predicted-{target}'] - omodels[tset][target]['data'][target][j][f'simulated-{target}']

# Avg Relative Error

In [ ]:
from numpy import *
from matplotlib.pyplot import *

In [ ]:
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.it'] = 'DejaVu Sans:italic'
rcParams['mathtext.bf'] = 'DejaVu Sans:italic:bold'


In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    print(f'original_100_on_{tset}')
    plt.figure(figsize=[20, 8])
    for target in ['COF', 'intercept']:
        plt.subplot(1, 2, 1 if target=='COF' else 2)
        val = list() # x axis
        err = list() # y axis 
        for j in emodels[tset][target][point]['data'][0][target]:
            if j!='r_square':
                val.append(omodels[tset][target]['data'][target][j][f'simulated-{target}'])
                err.append(omodels[tset][target]['ratio-to-simulated'][j])
        plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
        plt.ylabel('Relative Error')
        if target == 'COF':
            plt.ylim(0.7, 1.3)
            plt.xlim(0.085, 0.2)
        else:
            plt.ylim(0.5, 3)
            plt.xlim(0, 9)
        plt.axhline(y=1.0, linestyle='--', linewidth=3, color='k', alpha=0.2)
        plt.scatter(val, err, alpha=alpha)
    plot_name = f'original_100_on_{tset}'
    plt.savefig(fname=f'./plots/errors/avg/relative/{plot_name}.pdf')
    plt.show()
    plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in mpoints:
        print(f'm5050_{point}_on_{tset}')
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            val = list() # x axis
            err = list() # y axis 
            for j in m5models[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    val.append(m5models[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    err.append(m5models[tset][target][point]['ratio-to-simulated'][j])
            plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
            plt.ylabel('Relative Error')
            if target == 'COF':
                plt.ylim(0.7, 1.3)
                plt.xlim(0.085, 0.2)
            else:
                plt.ylim(0.5, 3)
                plt.xlim(0, 8)
            plt.axhline(y=1.0, linestyle='--', linewidth=3, color='k', alpha=0.2)
            plt.scatter(val, err, alpha=alpha)
        plot_name = f'm5050_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/errors/avg/relative/{plot_name}.pdf')
        plt.show()
        plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in mpoints:
        print(f'everything_{point}_on_{tset}')
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            val = list() # x axis
            err = list() # y axis 
            for j in emodels[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    val.append(emodels[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    err.append(emodels[tset][target][point]['ratio-to-simulated'][j])
            plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
            plt.ylabel('Relative Error')
            if target == 'COF':
                plt.ylim(0.7, 1.3)
                plt.xlim(0.085, 0.2)
            else:
                plt.ylim(0.5, 3)
                plt.xlim(0, 8)
            plt.axhline(y=1.0, linestyle='--', linewidth=3, color='k', alpha=0.2)
            plt.scatter(val, err, alpha=alpha)
        plot_name = f'everything_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/errors/avg/relative/{plot_name}.pdf')
        plt.show()
        plt.close()
        
        

# Avg Absolute Error

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    print(f'original_100_on_{tset}')
    plt.figure(figsize=[20, 8])
    for target in ['COF', 'intercept']:
        plt.subplot(1, 2, 1 if target=='COF' else 2)
        val = list() # x axis
        err = list() # y axis 
        for j in omodels[tset][target]['data'][target]:
            if j!='r_square':
                val.append(omodels[tset][target]['data'][target][j][f'simulated-{target}'])
                err.append(omodels[tset][target]['error'][j])
        plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
        plt.ylabel('Absolute Error')

        if target == 'COF':
            plt.ylim(-0.05, 0.05)
            plt.xlim(0.085, 0.20)
        else:
            plt.ylim(-3.5, 3.5)
            plt.xlim(0, 8)

        plt.axhline(y=0, linestyle='--', linewidth=3, color='k', alpha=0.3)
        plt.scatter(val, err, alpha=alpha)
    plot_name = f'original_100_on_{tset}'
    plt.savefig(fname=f'./plots/errors/avg/abs/{plot_name}.pdf')
    plt.show()
    plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in mpoints:
        print(f'm5050_{point}_on_{tset}')
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            val = list() # x axis
            err = list() # y axis 
            for j in m5models[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    val.append(m5models[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    err.append(m5models[tset][target][point]['error'][j])
            plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
            plt.ylabel('Absolute Error')
            
            if target == 'COF':
                plt.ylim(-0.05, 0.05)
                plt.xlim(0.085, 0.2)
            else:
                plt.ylim(-3.5, 3.5)
                plt.xlim(0, 8)
            plt.axhline(y=0, linestyle='--', linewidth=3, color='k', alpha=0.2)
            plt.scatter(val, err, alpha=alpha)
        plot_name = f'm5050_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/errors/avg/abs/{plot_name}.pdf')
        plt.show()
        plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in epoints:
        print(f'everything_{point}_on_{tset}')
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            val = list() # x axis
            err = list() # y axis 
            for j in emodels[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    val.append(emodels[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    err.append(emodels[tset][target][point]['error'][j])
            plt.xlabel(target if target=='COF' else r'$\mathbf{F_0}$, nN')
            plt.ylabel('Absolute Error')
            if target == 'COF':
                plt.ylim(-0.05, 0.05)
                plt.xlim(0.085, 0.2)
            else:
                plt.ylim(-3.5, 3.5)
                plt.xlim(0, 8)
            plt.axhline(y=0, linestyle='--', linewidth=3, color='k', alpha=0.2)
            plt.scatter(val, err, alpha=alpha)
        plot_name = f'everything_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/errors/avg/abs/{plot_name}.pdf')
        plt.show()
        plt.close()

# Sim vs Pred

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    plt.figure(figsize=[20, 8])
    for target in ['COF', 'intercept']:
        plt.subplot(1, 2, 1 if target=='COF' else 2)
        sim = list() # x axis
        prd = list() # y axis 
        for j in omodels[tset][target]['data'][target]:
            if j!='r_square':
                sim.append(omodels[tset][target]['data'][target][j][f'simulated-{target}'])
                prd.append(omodels[tset][target]['data'][target][j][f'predicted-{target}'])
        '''
        crr_matrix = np.corrcoef(prd, sim)
        crr_xy = crr_matrix[0, 1]
        r_square = crr_xy**2     
        '''
        u = sum([(prd[a] - sim[a])**2 for a in range(len(prd))])
        v = sum([(sim[a] - statistics.mean(sim))**2 for a in range(len(sim))])
        r_square = 1 - (u/v)
        
        if target == 'COF':
            plt.ylim(0.085, 0.2)
            plt.xlim(0.085, 0.2)
            plt.xlabel('COF (Predicted)')
            plt.ylabel('COF (Simulated)')
            plt.text(x=0.16, y=0.1, s=f'R$^2$={r_square:.3f}', fontsize=28)

        else:
            plt.ylim(0, 8)
            plt.xlim(0, 8)
            plt.xlabel(r'$\mathbf{F_0}$ (Predicted), nN')
            plt.ylabel(r'$\mathbf{F_0}$ (Simulated), nN')
            plt.text(x=5.5, y=1, s=f'R$^2$={r_square:.3f}', fontsize=28)

        xpoints = ypoints = plt.xlim()


        plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=False, scaley=False, alpha=0.2)

        bound_lines = 0.15 
        yuppers = [x*(1+bound_lines) for x in xpoints]
        ylowers = [x*(1-bound_lines) for x in xpoints]
        plt.plot(xpoints, yuppers, linestyle='--',
            color='k', lw=3, scalex=False,
            scaley=False, alpha=0.2)
        plt.plot(xpoints, ylowers, linestyle='--',
            color='k', lw=3, scalex=False,
            scaley=False, alpha=0.2)
        plt.scatter(prd, sim, alpha=alpha, marker='o')


    plot_name = f'original_100_on_{tset}'
    plt.savefig(fname=f'./plots/avg_pred_vs_sim/{plot_name}.pdf', dpi=500)
    plt.show()
    plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in mpoints:
        print(f'm5050_{point}_on_{tset}')
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            sim = list() # x axis
            prd = list() # y axis 
            for j in m5models[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    sim.append(m5models[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    prd.append(m5models[tset][target][point]['ave_data'][j])
                    
#             crr_matrix = np.corrcoef(prd, sim)
#             crr_xy = crr_matrix[0, 1]
#             r_square = crr_xy**2     

            u = sum([(prd[a] - sim[a])**2 for a in range(len(prd))])
            v = sum([(sim[a] - statistics.mean(sim))**2 for a in range(len(sim))])
            r_square = 1 - (u/v)
            
            if target == 'COF':
                plt.ylim(0.085, 0.2)
                plt.xlim(0.085, 0.2)
                plt.xlabel('COF (Predicted)')
                plt.ylabel('COF (Simulated)')
                plt.text(x=0.16, y=0.1, s=f'R$^2$={r_square:.3f}', fontsize=28)

            else:
                plt.ylim(0, 8)
                plt.xlim(0, 8)
                plt.xlabel(r'$\mathbf{F_0}$ (Predicted), nN')
                plt.ylabel(r'$\mathbf{F_0}$ (Simulated), nN')
                plt.text(x=5.5, y=1, s=f'R$^2$={r_square:.3f}', fontsize=28)

            xpoints = ypoints = plt.xlim()
            
            
            plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=False, scaley=False, alpha=0.2)

            bound_lines = 0.15 
            yuppers = [x*(1+bound_lines) for x in xpoints]
            ylowers = [x*(1-bound_lines) for x in xpoints]
            plt.plot(xpoints, yuppers, linestyle='--',
                color='k', lw=3, scalex=False,
                scaley=False, alpha=0.2)
            plt.plot(xpoints, ylowers, linestyle='--',
                color='k', lw=3, scalex=False,
                scaley=False, alpha=0.2)
            plt.scatter(prd, sim, alpha=alpha, marker='o')


        plot_name = f'm5050_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/avg_pred_vs_sim/{plot_name}.pdf', dpi=500)
        plt.show()
        plt.close()

In [ ]:
alpha=0.2
for tset in ['5050', '2575', 'everything']:
    for point in epoints:
        plt.figure(figsize=[20, 8])
        for target in ['COF', 'intercept']:
            plt.subplot(1, 2, 1 if target=='COF' else 2)
            sim = list() # x axis
            prd = list() # y axis 
            for j in emodels[tset][target][point]['data'][0][target]:
                if j!='r_square':
                    sim.append(emodels[tset][target][point]['data'][0][target][j][f'simulated-{target}'])
                    prd.append(emodels[tset][target][point]['ave_data'][j])
                    
#             crr_matrix = np.corrcoef(prd, sim)
#             crr_xy = crr_matrix[0, 1]
#             r_square = crr_xy**2     
            u = sum([(prd[a] - sim[a])**2 for a in range(len(prd))])
            v = sum([(sim[a] - statistics.mean(sim))**2 for a in range(len(sim))])
            r_square = 1 - (u/v)
            
            if target == 'COF':
                plt.ylim(0.085, 0.2)
                plt.xlim(0.085, 0.2)
                plt.xlabel('COF (Predicted)')
                plt.ylabel('COF (Simulated)')
                plt.text(x=0.16, y=0.1, s=f'R$^2$={r_square:.3f}', fontsize=28)

            else:
                plt.ylim(0, 8)
                plt.xlim(0, 8)
                plt.xlabel(r'$\mathbf{F_0}$ (Predicted), nN')
                plt.ylabel(r'$\mathbf{F_0}$ (Simulated), nN')
                plt.text(x=5.5, y=1, s=f'R$^2$={r_square:.3f}', fontsize=28)

            xpoints = ypoints = plt.xlim()
            
            
            plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=False, scaley=False, alpha=0.2)

            bound_lines = 0.15 
            yuppers = [x*(1+bound_lines) for x in xpoints]
            ylowers = [x*(1-bound_lines) for x in xpoints]
            plt.plot(xpoints, yuppers, linestyle='--',
                color='k', lw=3, scalex=False,
                scaley=False, alpha=0.2)
            plt.plot(xpoints, ylowers, linestyle='--',
                color='k', lw=3, scalex=False,
                scaley=False, alpha=0.2)
            plt.scatter(prd, sim, alpha=alpha, marker='o')


        plot_name = f'everything_{point}_on_{tset}'
        plt.savefig(fname=f'./plots/avg_pred_vs_sim/{plot_name}.pdf', dpi=500)
        plt.show()
        plt.close()